In [1]:
# Install required packages
!pip install -q -U transformers peft accelerate bitsandbytes trl huggingface_hub
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset
import torch
from google.colab import drive

# Mount Google Drive to access your dataset
drive.mount('/content/drive')

# Load your formatted dataset
dataset_path = "/content/drive/MyDrive/databricks_docs_instructions.json"
dataset = load_dataset('json', data_files=dataset_path, split='train')

# Print sample to verify
print("Sample dataset entry:")
print(dataset[0])

# Model configuration
model_name = "mistralai/Mistral-7B-v0.1"

# Quantization config for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token
tokenizer.padding_side = "right"  # Prevent overflow issues

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# Prepare model for PEFT training
model = prepare_model_for_kbit_training(model)

# LoRA configuration
peft_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

# Apply LoRA
model = get_peft_model(model, peft_config)

# Print trainable parameters
model.print_trainable_parameters()

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,  # Reduce if OOM errors occur
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=3,
    max_steps=-1,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    fp16=True,
    push_to_hub=False,  # Set to True if you want to push to HF Hub
    report_to="none",
)

Mounted at /content/drive


Generating train split: 0 examples [00:00, ? examples/s]

Sample dataset entry:
{'instruction': '"How can I use the `stack` function to generate a set of rows with specified values in Databricks SQL Databricks Runtime?"', 'input': '', 'output': "You can use the `stack` function in Databricks SQL Databricks Runtime to generate a set of rows with specified values by providing the number of rows and the values as arguments. The function separates the values into the specified number of rows. Here's an example:\n\n```sql\nSELECT 'hello', s.*, 'world' FROM stack(2, 1, 2, 3) AS s(first, second);\n```\n\nIn this example, the `stack` function generates two rows with the values 1, 2 and 3. The first column is named 'first' and the second column is named 'second'. The result of this query would be:\n\n```\nhello | first | second | world\n------+-------+--------+-------\nhello |     1 |      2 | world\nhello |     3 |   NULL | world\n```\n\nRemember that in Databricks SQL Databricks Runtime 12.2 LTS and above, you should invoke the `stack` function as a

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

trainable params: 13,631,488 || all params: 7,255,363,584 || trainable%: 0.1879


In [3]:
# First, let's properly preprocess the dataset
def format_dataset(example):
    # Create the text in the required format
    text = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    return {"text": text}

# Apply the formatting to the entire dataset
formatted_dataset = dataset.map(format_dataset)

Map:   0%|          | 0/5169 [00:00<?, ? examples/s]

In [ ]:
# Now create the trainer with the properly formatted dataset
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=formatted_dataset,  # Use the formatted dataset
)

# Start training
print("Starting training...")
trainer.train()

Converting train dataset to ChatML:   0%|          | 0/5169 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/5169 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5169 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/5169 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Starting training...


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.011200
20,0.959000
30,0.855000
40,0.936200
50,0.875200
60,0.938000
70,0.855700
80,0.925900
90,0.862200
100,0.877400


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=969, training_loss=0.7049081908660039, metrics={'train_runtime': 4313.5495, 'train_samples_per_second': 3.595, 'train_steps_per_second': 0.225, 'total_flos': 3.6498184641193574e+17, 'train_loss': 0.7049081908660039})

In [ ]:
# Save the model
output_dir = "/content/drive/MyDrive/databricks_docs_model"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Training complete! Model saved to {output_dir}")

Training complete! Model saved to /content/drive/MyDrive/databricks_docs_model
